# CS231n Winter 2016: Lecture 3
## Topics
- Linear Classification, 
- Optimization
- SVN

## Sources
- video: https://www.youtube.com/watch?v=qlLChbHhbg4
- original notes by Andrej Karpathy: http://cs231n.github.io/optimization-1/

In [1]:
from IPython.display import HTML
video_id = 'qlLChbHhbg4'
HTML(f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [2]:
import numpy as np

## Lost function _(Cost function, Objective)_

### Multiclass Support Vector Machine (SVN) loss
- based on Weston and Watkins 1999 version
- **TODO:** take a look on SVN [CS229 lecture notes](http://cs229.stanford.edu/notes/cs229-notes3.pdf)
- other alternatives: _kernels, duals, the SMO optimization algorithm, One-Vs-All (OVA), All-vs-All (AVA) strategy, Structured SVM._

SVN loss 
- [hinge loss](https://en.wikipedia.org/wiki/Hinge_loss) (max-margin loss). **TODO: what is it?**:
$$
L_i = \sum_{j \neq y_i} max(0, s_j - s_{y_{i}} + 1)
$$
where scores:
$$
s = f(x_i, W)
$$
`1` is margin
- squared hinge loss *L2-SVM* (sometimes could be used). **TODO: why?**
$$
L_i = \sum_{j \neq y_i} max(0, s_j - s_{y_{i}} + 1)^{2}
$$
- usually at initialization $W$ closes to $0$ (zero). In this case $s$ also would be close to $0$ and $L_i \approx number of classes - 1$

In [3]:
def l_i_vectorized(x, y, W):
    scores = W.dot(x)
    margins = np.maximum(0, scores - scores[y] + 1)
    margins[y] = 0 # or we could subtract 1 instead
    return np.sum(margins)

### Regularization
problem with this form of $W$ - that we could scale $W$ by factor (>1) and get the same loss result.
So we should use regularization:
$$
L_i = \frac{1}{N} \sum_{i=1}^{N} \sum_{j \neq y_i} max(0, f(x_i, W)_j - f(x_i, W)_{y_{i}} + 1) + \lambda R(W)
$$

in common use:
- **L2 regularization** $R(W) = \sum_k \sum_l W^{2}_{k,l}$ - tries to spread $W$ and take into account the most features (prefers smaller and more diffuse weight vectors)
- L1 regularization $R(W) = \sum_k \sum_l |W_{k,l}|$
- Elastic net (L1 + L2)  $R(W) = \sum_k \sum_l \beta W^{2}_{k,l} + |W_{k,l}|$
- Max norm regularization
- Dropout

$$
L = "data loss" + "regularization loss" = \frac{1}{N} \sum_{i}L_i + \lambda R(W)
$$
where N is number of training examples

### Softmax Classifier (Multinominal Logistic Regression)
alternative to SVN loss function.

we consider that scores is unnormalized log probability of the classes

$$
P(Y=k|X=x_i) = \frac{e^{s_k}}{\sum_{j} e^{s_j}}
$$
we use the log likelihood to maximize probability (cross-entropy loss):
$$
L_i = - log P(Y=k|X=x_i) = - log \frac{e^{s_{y_i}}}{\sum_j e^{s_{y_j}}} = - s_{y_i} + log \sum_j e^{s_{y_j}}
$$

*Property:*
- min = 0, max = infinity
- for small $W$, $L_i \approx - log (1/N)$
- for classifier with score maximum for the right classes if we will jiggle scores a little bit we likely to get the same SVN than Softmax. Robustness of SVN and Micromanagement of softmax.

*Related*
- [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)

### numerical stability
Dividing large numbers can be numerically unstable thus:
$$
\frac{e^{s_k}}{\sum_{j} e^{s_j}} = \frac{C e^{s_k}}{C \sum_{j} e^{s_j}} = \frac{e^{s_k + log C}}{\sum_{j} e^{s_j + log C}}
$$
A common chooice
$$
log C = - max_j f_j
$$

In [4]:
f = np.array([123, 456, 789]) # example with 3 classes and each having large scores
p = np.exp(f) / np.sum(np.exp(f)) # Bad: Numeric problem, potential blowup

print(f'without shifting: {p}')
# instead: first shift the values of f so that the highest number is 0:
f -= np.max(f) # f becomes [-666, -333, 0]
p = np.exp(f) / np.sum(np.exp(f)) # safe to do, gives the correct answer

print(f'with shifting: {p}')

without shifting: [  0.   0.  nan]
with shifting: [  5.75274406e-290   2.39848787e-145   1.00000000e+000]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


## Gradient of Softmax
TODO: add description. For example https://stackoverflow.com/questions/41663874/cs231n-how-to-calculate-gradient-for-softmax-loss-function
$$
\nabla_{w_{y_i}}L_i = 
$$

## Optimization
- *random search* - render random $W$ and check its loss function
- *gradient*
  - iterative refinement - update position by some delta on each iteration
  - numerical - approximate, very slow, but easy to write (we use it for debug/check)
  - analytical - exact, fast, but error-prone (use in practice)
- **step-size** (learning rate) - choose right value is very complex trick
- mini-batch gradient descent (only few samples - 32/64/128)

- **ME:** *we use physic analog for optimization step value (learning rate) problem here. Optization like throwing a ball down to the funnel. But if we wouldn't have friction or any other lost energy ball would bumping around and never would set to the lowest point, so what do we need here is a decrease of energy after each iteration.* And btw it is a common practice of decaying learning rate

## Practice before CNN popularity
- usually don't use raw pixels but extract features: histogram of colors (hue bins), HOG/SIFT/Texton/SSIM features
- bag of words of features

### ... to be continue [./lec4-computational-graph-backpropagation-neural-networks.ipynb](./lec4-computational-graph-backpropagation-neural-networks.ipynb)